In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
cd '/content/drive/My Drive/ComputerVision_project'

/content/drive/My Drive/ComputerVision_project


In [0]:
from __future__ import print_function, division
import numpy as np
import os
import torch
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

import json
from os import path

from skimage.io import imsave

import numpy as np

import os
import torch
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

import torch
import torchvision

import torch.nn as nn
import torch.nn.functional as F

import torch.nn as nn
import torch.nn.functional as F
import torch
import torchvision

import torch.nn as nn
import torch.nn.functional as F

import torch
 
from torch.distributions.multinomial import Multinomial
from torch.distributions.categorical import Categorical
from torch.distributions.uniform import Uniform
import torch.nn.functional as F


class Batcher():
    """Assemble a sequence of things into a sequence of batches."""
    def __init__(self, sequence, batch_size=16):
        self._batch_size = batch_size
        self._sequence = sequence
        self._idxs = np.arange(len(self._sequence))

    def __len__(self):
        return int(np.ceil(len(self._sequence) / self._batch_size))

    def __getitem__(self, i):
        if i >= len(self):
            raise IndexError("Index out of bounds")

        start = i*self._batch_size
        end = min(len(self._sequence), start+self._batch_size)
        data = [self._sequence[j] for j in self._idxs[start:end]]
        inputs = [d[0] for d in data]
        outputs = [d[1] for d in data]

        return self._stack(inputs), self._stack(outputs)

    def _stack(self, data):
        if data is None:
            return None

        if not isinstance(data[0], (list, tuple)):
            return np.stack(data)

        seq = type(data[0])
        K = len(data[0])
        data = seq(
            np.stack([d[k] for d in data])
            for k in range(K)
        )

        return data

    def on_epoch_end(self):
        np.random.shuffle(self._idxs)
        self._sequence.on_epoch_end()


class MNIST():
    """Load a Megapixel MNIST dataset."""
    def __init__(self, dataset_dir, train=True):
        """Passing the directory name to the JSON parameters, to load th parameters."""
        with open(path.join(dataset_dir, "parameters.json")) as f:
            self.parameters = json.load(f)

        filename = "train.npy" if train else "test.npy"
        N = self.parameters["n_train" if train else "n_test"]
        W = self.parameters["width"]
        H = self.parameters["height"]
        scale = self.parameters["scale"]

        self._high_shape = (H, W, 1)
        self._low_shape = (int(scale*H), int(scale*W), 1)
        self._data = np.load(path.join(dataset_dir, filename),allow_pickle=True)

    def __len__(self):
        return len(self._data)

    def __getitem__(self, i):
        if i >= len(self):
            raise IndexError()

        # Placeholders
        x_low = np.zeros(self._low_shape, dtype=np.float32).ravel()
        x_high = np.zeros(self._high_shape, dtype=np.float32).ravel()

        # Fill the sparse representations
        data = self._data[i]
        x_low[data[0][0]] = data[0][1]
        x_high[data[1][0]] = data[1][1]

        # Reshape to their final shape
        x_low = x_low.reshape(self._low_shape)
        x_high = x_high.reshape(self._high_shape)

        return [x_low, x_high], data[2]


class AttentionSaver():
    def __init__(self, output, att_model, data):
        self._att_path = path.join(output, "attention_{:03d}.png")
        self._patches_path = path.join(output, "patches_{:03d}_{:03d}.png")
        self._att_model = att_model
        (self._x, self._x_high), self._y = data[0]
        self._imsave(
            path.join(output, "image.png"),
            self._x[0, :, :, 0]
        )

    def on_epoch_end(self, e, logs):
        att, patches = self._att_model.predict([self._x, self._x_high])
        self._imsave(self._att_path.format(e), att[0])
        np.save(self._att_path.format(e)[:-4], att[0])
        for i, p in enumerate(patches[0]):
            self._imsave(self._patches_path.format(e, i), p[:, :, 0])

    def _imsave(self, filepath, x):
        x = (x*65535).astype(np.uint16)
        imsave(filepath, x, check_contrast=False)

class MegaMnistDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, dataset_dir, train ):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.mnist = MNIST(dataset_dir=dataset_dir, train=train)

    def __len__(self):
        return len(self.mnist)

    def __getitem__(self, idx):
        [x_low, x_high], response = self.mnist[idx]
        x_low, x_high = np.transpose(x_low, (2,0,1)), np.transpose(x_high, (2,0,1))
        x_low = torch.Tensor(x_low)
        x_high = torch.Tensor(x_high)
        response = torch.Tensor(response)
        return [x_low, x_high], response

class SampleSoftmaxLayer(nn.Module):
    def __init__(self, squeeze_channels = False, smooth=0, **kwargs):
        self.squeeze_channels = squeeze_channels
        self.smooth = torch.tensor(smooth, device = device, requires_grad=True)
        super(SampleSoftmaxLayer, self).__init__()
    
    def forward(self, x):
        # Apply softmax to the whole x (per sample)
        input_shape = x.shape
        x = x.view(input_shape[0], -1)
        x = F.softmax(x)
        # smooth the distribution ? what distribution
        if 0 < self.smooth < 1:
            x = x * ( 1 - self.smooth)
            x = x + self.smooth / torch.tensor(x.shape[1], device = device, dtype= torch.float32, requires_grad= True)

        # Finally reshape to the original shape
        x = x.view(input_shape)

        if self.squeeze_channels:
            # channels are at the beginning now
            channels = 1
            x = torch.squeeze(x, channels)
        return x

class L2NormalizeLayer(nn.Module):
    def __init__(self, axis = -1, **kwargs):
        self.axis = axis
        super(L2NormalizeLayer, self).__init__(**kwargs)

    def forward(self, x):
        # https://discuss.pytorch.org/t/how-to-normalize-embedding-vectors/1209/7
        norm = x.norm(p = 2, dim = self.axis, keepdim= True)
        response =  x.div(norm).detach()
        response[torch.isnan(response)] = 0
        response[torch.isinf(response)] = 0
        return response

def _sample_with_replacement(logits):
    """Sample with replacement using the tensorflow op."""
    return Categorical(logits=logits)
 
def _sample_without_replacement(logits, n_samples):
    """Sample without replacement using the Gumbel-max trick.
 
    See lips.cs.princeton.edu/the-gumbel-max-trick-for-discrete-distributions/
    """
    z = -torch.log(-torch.log(torch.rand(logits.shape, device = device)))
    # returns the indices of the top n samples
    return torch.topk(logits+z, k=n_samples, dim=1)[1]
 
def gather_nd(params, indices, name=None):
    '''
    the input indices must be a 2d tensor in the form of [[a,b,..,c],...], 
    which represents the location of the elements.
    '''
    
    indices = indices.t().long()
    ndim = indices.size(0)
    idx = torch.zeros_like(indices[0], device = device, requires_grad = True).long()
    m = 1
    
    for i in range(ndim)[::-1]:
        idx += indices[i] * m 
        m *= params.size(i)
    
    return torch.take(params, idx, )
 
def unravel_index(index, shape):
    #return torch.tensor(np.unravel_index(index.cpu().numpy(), shape), device = device)
    
    out = []
    for dim in reversed(shape):
        out.append(index % dim)
        index = index // dim
    return torch.cat(out[::-1], 0)
 
def sample(n_samples, attention, sample_space, replace=False,
           use_logits=False):
    """Sample from the passed in attention distribution.
 
    Arguments
    ---------
        n_samples: int, the number of samples per datapoint
        attention: tensor, the attention distribution per datapoint (could be
                   logits or normalized)
        sample_space: This should always equal K.shape(attention)[1:]
        replace: bool, sample with replacement if set to True (defaults to
                 False)
        use_logits: bool, assume the input is logits if set to True (defaults
                    to False)
    """
    
    # Make sure we have logits and choose replacement or not
    input_shape = attention.shape
    logits = attention if use_logits else torch.log(attention)
    sampling_function = (
        _sample_with_replacement if replace
        else _sample_without_replacement
    )
    
    # Flatten the attention distribution and sample from it
    logits = logits.view(-1, torch.prod(torch.tensor(sample_space)))
    samples = sampling_function(logits, n_samples)
    good_samples = samples
    
 
    
    # Unravel the indices into sample_space
    batch_size = attention.shape[0]
    n_dims = torch.tensor(sample_space).shape[0]
    samples =  unravel_index(samples.view((-1,)), sample_space)
    
    samples = torch.t(samples).view((batch_size, n_samples, n_dims))
 
    # Concatenate with the indices into the batch dimension in order to gather
    # the attention values
    batch_indices = (
        torch.arange(0,batch_size, device=device).view((-1, 1, 1)) *
        torch.ones((1, n_samples, 1), device=device)
    )
    
    indices = torch.cat([batch_indices.long(), samples.long()], -1)
    
    wow = attention.view(input_shape[0], 174 * 174)
    sampled_attention = torch.gather(wow, 1, good_samples)
 
    # Gather the attention
    #sampled_attention = attention.masked_select(indices)
 
    return samples, sampled_attention


class AttentionModel(nn.Module):
  def __init__(self):
    super(AttentionModel, self).__init__()
    self.conv1 = nn.Conv2d(1, 8, kernel_size =3,)
    self.conv2 = nn.Conv2d(8,8, kernel_size = 3, )
    self.conv3 = nn.Conv2d(8, 1, kernel_size=3, )
    self.sample_softmax = SampleSoftmaxLayer(squeeze_channels=True, smooth=1e-5)
  
  def forward(self, x):
    x = torch.tanh(self.conv1(x))
    x = torch.tanh(self.conv2(x))
    x = self.conv3(x)
    response =  self.sample_softmax(x)
    return response

class FeatureModel(nn.Module):
  def __init__(self):
    super(FeatureModel, self).__init__()
    self.conv1 = nn.Conv2d(1, 32 ,kernel_size=7)
    self.conv2 = nn.Conv2d(32, 32 ,kernel_size=3)
    self.conv3 = nn.Conv2d(32, 32 ,kernel_size=3)
    self.conv4 = nn.Conv2d(32, 32 ,kernel_size=3)
    self.avg_pooling = nn.AdaptiveMaxPool2d(1)
    self.l2_layer = nn.BatchNorm2d(32)
  
  def forward(self, x):
    x = torch.relu(self.conv1(x))
    x = torch.relu(self.conv2(x))
    x = torch.relu(self.conv3(x))
    x = torch.relu(self.conv4(x))
    x = self.avg_pooling(x)
    return self.l2_layer(x)


class SamplePatches(nn.Module):
    def __init__(self,n_patches, patch_size, receptive_field =0, replace= False, use_logits = False, **kwargs):
        self._n_patches = n_patches
        self._patch_size = patch_size
        self._receptive_field = receptive_field
        self._replace = replace
        self._use_logits = use_logits

        super(SamplePatches, self).__init__()
    
    def forward(self, x_low, x_high, attention):
        sample_space = attention.shape[1:]
        samples, sampled_attention = sample(
            self._n_patches,
            attention,
            sample_space,
            replace=self._replace,
            use_logits= self._use_logits
        )

        offsets = torch.zeros_like(samples, device = device, dtype=torch.float32, requires_grad = True)
        if self._receptive_field > 0:
            offsets = offsets + self._receptive_field / 2
        patches, _ = FromTensors([x_low, x_high], None).patches(
            samples,
            offsets,
            sample_space,
            torch.tensor(x_low.shape[2:]) - self._receptive_field,
            self._patch_size,
            0,
            1
        )
        return [patches, sampled_attention]


class TotalReshape(nn.Module):
  def __init__(self, shape):
    self._shape = shape
  
  def forward(self, x):
    return x.view(self._shape)


def expand_many(x, axes):
  for ax in axes:
    x = x.unsqueeze(ax)
  return x

class FromTensors():
  def __init__(self, xs, y):
    self._xs = xs
    self._y = y
  
  def targets(self):
    return self._y
  
  def inputs(self):
    return []

  def data(self, level):
    return self._xs[level]

  def _scale(self, shape_from, shape_to):
    shape_from = shape_from
    shape_to = shape_to
    return shape_to / shape_from
  
  def _shape(self, level):
    x = self._xs[level]
    int_shape = x.shape[2:]
    
    if not any(s is None for s in int_shape):
      return int_shape
    return x.shape[2:]

  def patches(self, samples, offsets, sample_space, previous_patch_size, patch_size, fromlevel, tolevel):
    # Make sure that everything is a pixel
    sample_space = torch.tensor(sample_space, device = device, dtype=torch.float32, requires_grad=True)
    previous_patch_size = torch.tensor(previous_patch_size, device = device, dtype=torch.float32, requires_grad=True)
    patch_size = torch.tensor(patch_size, device = device, dtype=torch.float32, requires_grad=True)  
    shape_from = torch.tensor(self._shape(fromlevel), device = device, dtype=torch.float32, requires_grad=True)
    shape_to = torch.tensor(self._shape(tolevel), device = device, dtype=torch.float32, requires_grad=True)

    # compute the scales
    scale_samples = self._scale(sample_space, shape_to)
    scale_offsets = self._scale(shape_from, shape_to)
    # compute the scales
    space_available = previous_patch_size  * scale_offsets
    steps = space_available / sample_space

    # steps is the offset per pixel 
    offsets = (
      offsets.float() * expand_many(scale_offsets.float(), [0, 0]) +
      samples.float() * expand_many(steps, [0, 0]) +
      expand_many(steps / torch.tensor(2, device = device, dtype=torch.float32, requires_grad=True), [0, 0]) -
      expand_many(patch_size / torch.tensor(2, device = device, dtype=torch.float32, requires_grad=True), [0, 0])
    )

    patches = extract_patches(
        self._xs[tolevel],
        offsets,
        patch_size
    )

    return patches, offsets

class SamplingWithReplacement(torch.autograd.Function):
  @staticmethod
  def forward(ctx, weights, attention, features):
    ctx.save_for_backward(weights, attention, features)
    wf = expand_many(weights, [-1] * (features.ndim - 2) )
    F = torch.sum(wf * features, axis = 1)
    return F
  
  @staticmethod
  def backward(ctx, grad_output):
    weights, attention, features = ctx.saved_tensors
    grad_output.unsqueeze_(-1)
    
    # gradient of attention
    ga = grad_output * features
    ga = torch.sum(ga, axis = list(range(2, torch.ndim(ga))))
    ga = ga * weights / attention
    
    # geadient of features
    gf = wf * grad_output
    return [None, ga, gf]

def _SamplingWithoutReplacement(weights, attention, features):
  axes = [-1] * ((features).ndimension() - 2 )
  wf = expand_many(weights, axes)
  af = expand_many(attention, axes)
  pm = 1 - torch.cumsum(attention, 1)
  pmf = expand_many(pm, axes)
  Fa = af * features
  Fpm = pmf * features
  Fa_cumsum = torch.cumsum(Fa, axis = 1)
  F_estimater = Fa_cumsum + Fpm
  F = torch.sum(wf * F_estimater, axis =1)
  return F

class SamplingWithoutReplacement(torch.autograd.Function):
  @staticmethod
  def forward(ctx, weights, attention, features):
    ctx.save_for_backward(weights, attention, features)

    axes = [-1] * ((features).ndimension() - 2 )
    wf = expand_many(weights, axes)
    af = expand_many(attention, axes)
    pm = 1 - torch.cumsum(attention, 1)
    pmf = expand_many(pm, axes)
    Fa = af * features
    Fpm = pmf * features
    Fa_cumsum = torch.cumsum(Fa, axis = 1)
    F_estimater = Fa_cumsum + Fpm
    F = torch.sum(wf * F_estimater, axis =1)
    return F
  
  @staticmethod
  def backward(ctx, grad_output):
    weights, attention, features = ctx.saved_tensors
    axes = [-1] * ((features).ndimension() - 2 )
    af = expand_many(attention, axes)
    wf = expand_many(weights, axes)
    pm = 1 - torch.cumsum(attention, 1)
    pmf = expand_many(pm, axes)
    Fa = af * features
    Fpm = pmf * features
    Fa_cumsum = torch.cumsum(Fa, axis = 1)
    F_estimator = Fa_cumsum + Fpm
    F = torch.sum(wf * F_estimator, axis =1)

    axes = [-1] * ((features).ndimension() - 2 )
    N = attention.shape[1]
    probs = attention / pm
    probsf = expand_many(probs, axes)
    grad_output = torch.unsqueeze(grad_output, 1)
    ga1 = F_estimator / probsf
    ga2 = (torch.cumsum(features, axis =1) - expand_many(torch.arange(N, device=device, dtype=torch.float32), [0] + axes) * features )
    ga = grad_output * (ga1 + ga2)
    ga = ga.sum(axis = list(range(2, ga.ndimension())))
    ga = ga * weights

    gf = expand_many(torch.arange(N-1,-1,-1, device=device).float(), [0] + axes)
    gf = pmf + gf * af
    gf = wf * gf
    gf = gf * grad_output

    return None, ga, gf

def expected(attention, features, replace = False, weights = None):
  if weights is None:
    weights = torch.ones_like(attention) / torch.tensor(attention.shape[1], dtype=torch.float32)
  
  E = SamplingWithReplacement.apply if replace else SamplingWithoutReplacement.apply
  return E(weights, attention, features)

class BhasRegularizer(nn.Module):
  def __init__(self, strength, eps):
    super(BhasRegularizer, self).__init__()
    self.strength = torch.tensor(strength, dtype = torch.float32, device= device, requires_grad=True)
    self.eps = torch.tensor(eps, dtype = torch.float32, device= device, requires_grad=True)
  
  def forward(self, x):
    logx = torch.log(x + self.eps)
    return self.strength * torch.sum(x*logx) / torch.tensor(x.shape[0], dtype = torch.float32, device= device, requires_grad=True)

class ExpectationLayer(nn.Module):
  def __init__(self, replace = False):
    self._replace = replace
    super(ExpectationLayer, self).__init__()

  def forward(self, x):
    features, sampled_attention = x
    return expected(sampled_attention, features, replace = self._replace)


class AttentionSamplingModel(nn.Module):
  def __init__(self, output_size, patch_size, n_patches, replace= False, attention_regularizer=None, receptive_field = 0):
    super(AttentionSamplingModel, self).__init__()
    channels = dims = 1
    # other hyper params used in the network
    self.patch_size = patch_size
    self.n_patches = n_patches
    self.replace = replace
    self.attention_regularizer = BhasRegularizer(1,0.0001)
    # defining layers of repsonse
    self.attention = AttentionModel()
    self.feature = FeatureModel()
    self.layer = nn.Linear(32,output_size)
    self.samplePatchesLayer = SamplePatches( patch_size=patch_size, n_patches=n_patches, replace= False, attention_regularizer=None, receptive_field = 0)
    self.totalReshape1 = ((-1, channels, *patch_size,))
    self.totalReshape2 = ((-1, patch_size, dims))
    self.expectation = ExpectationLayer(replace )
    # make sure certain params are filled
    if receptive_field is None:
        raise NotImplementedError(("Receptive field is not implemented yet"))
    # make sure that patch values are filled
    if patch_size is None:
        # we need the patch size to be present
        # if needed we will add code to cover this functionality
        raise NotImplementedError("Patch size cannot be None")
  
  def forward(self, x):
    # we expect that the input values will be a list of items
    x_low, x_high = x
    attention_map = self.attention(x_low)

    # place holder to add attention regularizer
    patches, sampled_attention = self.samplePatchesLayer(x_low, x_high, attention_map)
    # compute features of the sampled patches
    channels = patches.shape[-1]
    #x.view(self._shape)
    patches_flat = patches.view(self.totalReshape1)
    patches_flat = patches_flat.clone().detach().requires_grad_(True)
    #print('Patches flat', patches_flat, torch.isnan(patches_flat).any())
    patch_features = self.feature(patches_flat)
    dims = patch_features.shape[-3]
    patch_features = patch_features.view((-1, self.n_patches, dims))
    #print('Patch features ', patch_features)
    sample_features = self.expectation([patch_features, sampled_attention])
    #print('sample features', sample_features)
    response =  F.softmax(self.layer(sample_features))
    return response, self.attention_regularizer(attention_map)
    return [sample_features, attention_map, patches]

def _p(x, name=None):
    shape = None
    if x.name:
        name = x.name
    if x.shape:
        shape = x.shape
        print('Printing value of {} with shape {} :  {}'.format(name, shape, x))

def extract_patches(x, offsets, size):
    batch_size = x.shape[0]
    sample_sizes = offsets.shape[1]
    actual_images = x.squeeze(1)
    all_patches = []
    size_int = size.clone().int()
    response = torch.ones((batch_size, sample_sizes, size_int[0].item(), size_int[1].item()), device = device, requires_grad=True)
    for i in range(batch_size):
        patches = []
        for sample in range(sample_sizes):
            left, top = offsets[i][sample][0].item() , offsets[i][sample][1].item()
            left = max(0, left)
            top = max(top, 0)
            top = min(top, 1450)
            left = min(left, 1450)
            right, bottom = left + size[0].item() , top + size[1].item()
            left, bottom, right, top = list(map(int, [left, bottom, right, top]))
            patch = (actual_images[i][left: right,top: bottom])
            response[i][sample] = patch
    response = response.unsqueeze(-1)
    return response

def CategoricalCrossEntropyLoss(inputs, target):
    _, labels = target.max(dim=1)
    return F.cross_entropy(inputs, labels)
def plot_grad_flow(named_parameters):
    ave_grads = []
    layers = []
    for n, p in named_parameters:
        if(p.requires_grad) and ("bias" not in n):
            layers.append(n)
            ave_grads.append(p.grad.abs().mean())
    plt.plot(ave_grads, alpha=0.3, color="b")
    plt.hlines(0, 0, len(ave_grads)+1, linewidth=1, color="k" )
    plt.xticks(range(0,len(ave_grads), 1), layers, rotation="vertical")
    plt.xlim(xmin=0, xmax=len(ave_grads))
    plt.xlabel("Layers")
    plt.ylabel("average gradient")
    plt.title("Gradient flow")
    plt.grid(True)

import torch.optim as optim

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

awesome = MegaMnistDataset('.', True)
train_datalaoder = torch.utils.data.DataLoader(awesome, batch_size=20, shuffle=True, num_workers=5)
net = AttentionSamplingModel(
    output_size = 10,
    patch_size = (50,50),
    n_patches = 20,
    replace = False
)
net = net.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum = 0.9)
for epoch in range(250):
    running_loss = 0.0
    for i, data in enumerate(train_datalaoder, 0):
        inputs, labels = data
        inputs, labels = [inputs[0].to(device), inputs[1].to(device)], labels.to(device)
        optimizer.zero_grad()
        outputs, _loss = net(inputs)
        loss = CategoricalCrossEntropyLoss(outputs, labels)
        good_loss  = loss + _loss
        good_loss.backward()
        plot_grad_flow(net.named_parameters())
        optimizer.step()
        print('Epoch: ', epoch, ' Step: ', i, ' total ', len(train_datalaoder), 'Loss: ', loss)  

print('Finished training')
PATH = './amazing2_cifar_net.pth'
torch.save(net.state_dict(), PATH)

response = main()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:187: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:422: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:606: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch:  0  Step:  0  total  250 Loss:  tensor(2.3031, device='cuda:0', grad_fn=<NllLossBackward>)
Epoch:  0  Step:  1  total  250 Loss:  tensor(2.3021, device='cuda:0', grad_fn=<NllLossBackward>)
Epoch:  0  Step:  2  total  250 Loss:  tensor(2.2993, device='cuda:0', grad_fn=<NllLossBackward>)
Epoch:  0  Step:  3  total  250 Loss:  tensor(2.3026, device='cuda:0', grad_fn=<NllLossBackward>)
Epoch:  0  Step:  4  total  250 Loss:  tensor(2.3076, device='cuda:0', grad_fn=<NllLossBackward>)
Epoch:  0  Step:  5  total  250 Loss:  tensor(2.3042, device='cuda:0', grad_fn=<NllLossBackward>)
Epoch:  0  Step:  6  total  250 Loss:  tensor(2.3030, device='cuda:0', grad_fn=<NllLossBackward>)
Epoch:  0  Step:  7  total  250 Loss:  tensor(2.3067, device='cuda:0', grad_fn=<NllLossBackward>)
Epoch:  0  Step:  8  total  250 Loss:  tensor(2.3015, device='cuda:0', grad_fn=<NllLossBackward>)
Epoch:  0  Step:  9  total  250 Loss:  tensor(2.2981, device='cuda:0', grad_fn=<NllLossBackward>)
Epoch:  0  Step:  10